In [64]:
from sklearn import svm
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, zero_one_loss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Lectura y normalización de data

In [65]:
scaler = MinMaxScaler()

dataset = pd.read_csv('./data/Cardiotocographic-Training.csv')
y = dataset.CLASE.to_numpy()
X = dataset.drop('CLASE', axis=1).to_numpy()
X = scaler.fit_transform(X)

## Bootsrap

In [61]:
def bootstrap(X, y, model, k, c, g):
    indices = np.array([i for i in range (len(X))])
    precisions = []
    recalls = []
    f1s = []
    aucs = []
    errors = []
    for i in range(k):
        train_index = resample(indices, n_samples=k, replace=True)
        test_index = np.array([j for j in indices if j not in train_index])
        
        x_train, y_train = X[train_index], y[train_index]
        x_test, y_test = X[test_index], y[test_index]
        
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        y_pred_auc = model.predict_proba(x_test)

        errors.append(zero_one_loss(y_test, y_pred))

        precision = precision_score(y_test, y_pred, average='micro') # micro porque toma en cuenta el desbalanceaminto de clases
        precisions.append(precision)

        recall = recall_score(y_test, y_pred, average='micro')
        recalls.append(recall)

        f1 = f1_score(y_test, y_pred, average='micro')
        f1s.append(f1)

        auc = roc_auc_score(y_test, y_pred_auc, multi_class='ovr')
        aucs.append(auc)
    return [np.mean(errors), np.var(errors), c, g, precisions, recalls, f1s, aucs, errors]

## K-Fold Cross-Validation

In [49]:
def k_fold(X, y, model, k, c, g):
    skf = KFold(n_splits=k, shuffle=True, random_state=42)
    precisions = []
    recalls = []
    f1s = []
    aucs = []
    errors = []
    for train_index, test_index in skf.split(X, y):
        x_train, y_train = X[train_index], y[train_index]
        x_test, y_test = X[test_index], y[test_index]

        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        y_pred_auc = model.predict_proba(x_test)

        errors.append(zero_one_loss(y_test, y_pred))

        precision = precision_score(y_test, y_pred, average='micro') # micro porque toma en cuenta el desbalanceaminto de clases
        precisions.append(precision)

        recall = recall_score(y_test, y_pred, average='micro')
        recalls.append(recall)

        f1 = f1_score(y_test, y_pred, average='micro')
        f1s.append(f1)

        auc = roc_auc_score(y_test, y_pred_auc, multi_class='ovr')
        aucs.append(auc)
    return [np.mean(errors), np.var(errors), c, g, precisions, recalls, f1s, aucs, errors]


### Tuneo de modelo SVM con bootstrap

In [ ]:
Cs = [1, 10, 100]
gammas = [0.1, 0.01, 0.001]

results_svm_bootstrap = []

k = int(len(X)/10)

for c in Cs:
    for gama in gammas:
        results_svm_bootstrap.append(bootstrap(X, y, svm.SVC(kernel='linear', C=c, gamma=gama, decision_function_shape='ovr', probability=True), k, c, gama))

# Valores de bias y varianza del respectivo hiperparametro
for r in results_svm_bootstrap:
    print("c y g: ", r[2], r[3], "\nbias: ", r[0], "\nvarianza: ", r[1])

### Valores de metricas del mejor modelo SVM con Bootstrap

In [ ]:
print("precision: ", np.mean(results_svm_bootstrap[4][4]))
print("recall: ", np.mean(results_svm_bootstrap[4][5]))
print("f1: ", np.mean(results_svm_bootstrap[4][6]))
print("auc: ", np.mean(results_svm_bootstrap[4][7]))

In [ ]:
#clf = svm.SVC(kernel='linear', C=10, gamma=0.001, decision_function_shape='ovr', probability=True) # C es penalización por los valores fuera de la clasificación
# gamma controla la distancia de influencia de un punto de entrenamiento, valor pequeño indica que el radio de distancia es más grande, por lo que mas puntos se agrupan correctamente.
# valora alto implica que el radio se reduce y los puntos deben estar más cerca entre ellos para ser considerados del mismo grupo
# precisions, recalls, f1s, aucs, errors = k_fold(X, y, clf, 10)

### Tuneo del modelo SVM con K-Fold Cross-Validation

In [ ]:
Cs = [1, 10, 100]
gammas = [0.1, 0.01, 0.001]

results_svm_k = []

for c in Cs:
    for gama in gammas:
        results_svm_k.append(k_fold(X, y, svm.SVC(kernel='linear', C=c, gamma=gama, decision_function_shape='ovr', probability=True), 10, c, gama))

# Valores de bias y varianza del respectivo hiperparametro
for r in results_svm_k:
    print("c y g: ", r[2], r[3], "\nbias: ", r[0], "\nvarianza: ", r[1])

### Valores de metricas del mejor modelo SVM con K-Fold

In [ ]:
print("precision: ", np.mean(results_svm_k[4][4]))
print("recall: ", np.mean(results_svm_k[4][5]))
print("f1: ", np.mean(results_svm_k[4][6]))
print("auc: ", np.mean(results_svm_k[4][7]))

### Tuneo del modelo logística con K-Fold Cross-Validation

In [ ]:
results_log_k = []
penalty = ['l1', 'l2']
cs = [10, 1, 0.1]

for p in penalty:
    for c in cs:
        results_log_k.append(k_fold(X, y, LogisticRegression(penalty=p, C=c, multi_class='ovr', solver='liblinear', max_iter=1000), 10, p, c))

for r in results_log_k:
   print("penalty y c: ", r[2], r[3], "\nbias: ", r[0], "\nvarianza: ", r[1])

### Valores de métricas del mejor modelo logística con K-Fold

In [ ]:

print("precision: ", np.mean(results_log_k[4][4]))
print("recall: ", np.mean(results_log_k[4][5]))
print("f1: ", np.mean(results_log_k[4][6]))
print("auc: ", np.mean(results_log_k[4][7]))

### Tuneo del modelo logística con Bootstrap

In [ ]:
results_log_bootstrap = []
penalty = ['l1', 'l2']
cs = [10, 1, 0.1]

k = int(len(X)/10)

for p in penalty:
    for c in cs:
        results_log_bootstrap.append(bootstrap(X, y, LogisticRegression(penalty=p, C=c, multi_class='ovr', solver='liblinear', max_iter=1000), k, p, c))

for r in results_log_bootstrap:
   print("penalty y c: ", r[2], r[3], "\nbias: ", r[0], "\nvarianza: ", r[1])